In [3]:
import os
from flask import Flask, jsonify
from supabase import create_client, Client
from time import sleep

app = Flask(__name__)

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

TABLE_NAME = "v2_federato_amplitude_data"

In [ ]:
def get_terminal_events(user_id):
    """Calculate the top 5 terminal events (most common session-ending events) for a given user."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time, session_id")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Group events by session
        sessions = {}
        for event in events_response.data:
            session_id = event["session_id"]
            if session_id not in sessions:
                sessions[session_id] = []
            sessions[session_id].append(event["event_type"])
        
        # Count terminal events
        terminal_event_counts = {}
        total_sessions = len(sessions)
        
        for session_events in sessions.values():
            if session_events:  # Make sure session has events
                last_event = session_events[-1]
                terminal_event_counts[last_event] = terminal_event_counts.get(last_event, 0) + 1
        
        # Filter for events that occurred at least twice as terminal events
        terminal_event_counts = {
            event: count 
            for event, count in terminal_event_counts.items() 
            if count >= 2
        }
        
        # If no events occurred twice as terminal events, return empty list
        if not terminal_event_counts:
            return []
        
        # Calculate terminal rates (times event was terminal / total sessions)
        terminal_rates = {
            event: count / total_sessions 
            for event, count in terminal_event_counts.items()
        }
        
        # Get top 5 terminal events by frequency
        top_churn_events = sorted(
            [(event, rate) for event, rate in terminal_rates.items()],
            key=lambda x: x[1],
            reverse=True  # Sort in descending order
        )[:5]
        
        # Format results
        return [{
            "event": event,
            "terminal_rate": rate,
            "count": terminal_event_counts[event]
        } for event, rate in top_churn_events]
        
    except Exception as e:
        print(f"Error calculating terminal events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top terminal events
def update_all_users_terminal_events():
    try:
        page_size = 1000
        last_user_id = 0
        
        while True:
            try:
                # Get next batch of users
                users_response = supabase.table("user_table")\
                    .select("user_id")\
                    .gt("user_id", last_user_id)\
                    .order("user_id")\
                    .limit(page_size)\
                    .execute()
                
                # If no more users, break
                if not users_response.data:
                    break
                
                # Process each user in this batch
                for user in users_response.data:
                    user_id = user["user_id"]
                    terminal_events = get_terminal_events(user_id)
                    
                    # Update the user's record with their top terminal events
                    supabase.table("user_table")\
                        .update({"top_churn_events": terminal_events})\
                        .eq("user_id", user_id)\
                        .execute()
                    
                    print(f"Updated terminal events for user {user_id}")
                    last_user_id = user_id
                
                print(f"Completed batch up to user {last_user_id}")
                sleep(1)

            except Exception as batch_error:
                print(f"Batch error: {batch_error}")
                sleep(5)  # Longer sleep on error
                continue
        
        print("Completed updating all users' terminal events")
        
    except Exception as e:
        print(f"Error updating terminal events: {e}")

# Execute the update
update_all_users_terminal_events()

Updated terminal events for user 1
Updated terminal events for user 2
Updated terminal events for user 3
Updated terminal events for user 4
Updated terminal events for user 5
Updated terminal events for user 6
Updated terminal events for user 7
Updated terminal events for user 8
Updated terminal events for user 9
Updated terminal events for user 10
Updated terminal events for user 11
Updated terminal events for user 12
Updated terminal events for user 13
Updated terminal events for user 14
Updated terminal events for user 15
Updated terminal events for user 16
Updated terminal events for user 17
Updated terminal events for user 18
Updated terminal events for user 19
Updated terminal events for user 20
Updated terminal events for user 21
Updated terminal events for user 22
Updated terminal events for user 23
Updated terminal events for user 24
Updated terminal events for user 25
Updated terminal events for user 26
Updated terminal events for user 27
Updated terminal events for user 28
U